In [1]:
import cv2 as cv
import numpy as np
import os
import math

In [2]:
#to resize an image
def resize(img, scale_percent):
#     scale_percent = 400 # percent of original size
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
  
    # resize image
    resized = cv.resize(img, dim, interpolation = cv.INTER_AREA)
    return resized

In [3]:
frames = os.listdir(r"D:\Not yours\NU\Master's\2nd semester\Introduction to image processing\Project\Video")



for i in  range(len(frames)):
    img = cv.imread(r"D:\Not yours\NU\Master's\2nd semester\Introduction to image processing\Project\Video\frame" + str(i) + ".jpg")
    img = resize(img, 50)
    cv.imshow("traffic", img)
    key = cv.waitKey(25)
    if key & 0xFF == ord('q'):
        break
cv.destroyAllWindows()

In [4]:
def Car_detection(original_1, original_2, resize_factor = 100 ,x1 = 100, y1 = 150, width = 400, height = 100):
    ############### Car detection ########################
#     original_1 = cv.imread(r"D:\Not yours\NU\Master's\2nd semester\Introduction to image processing\Project\source_code\frames\\"+ frames[i])
    frame1 = cv.cvtColor(original_1, cv.COLOR_BGR2GRAY)
#     frame1 = cv.GaussianBlur(frame1, (3,3), 0 )
#     frame1 = resize(frame1, resize_factor)
#     original_1 = resize(original_1, resize_factor)

#     original_2 = cv.imread(r"D:\Not yours\NU\Master's\2nd semester\Introduction to image processing\Project\source_code\frames\\"+ frames[i+1])
    frame2 = cv.cvtColor(original_2, cv.COLOR_BGR2GRAY)
#     frame2 = cv.GaussianBlur(frame2, (3,3), 0 )
#     frame2 = resize(frame2, resize_factor)
#     original_2 = resize(original_2, resize_factor)


    #diff bet two frames
    diff = cv.absdiff(frame2, frame1)

    #thresholding
    _, thresh = cv.threshold(diff, 50, 255, cv.THRESH_BINARY)

    #Dilation
    dilation_kernel = np.ones((3,3), np.uint8)
    dilated_img = cv.dilate(thresh, dilation_kernel, iterations = 5)
    
    
    ############################################################################
#     kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (3, 3))
#     closing = cv.morphologyEx(dilated_img, cv.MORPH_CLOSE, kernel) 
#     Image.fromarray(closing)
    
    #########################################################################################################
    #find the contours in the image
    # def get_contours(img, start_point = (0,0), width = img.shape[1], height = img.shape[0]):

    contours, hierarchy = cv.findContours(dilated_img, cv.RETR_TREE,cv.CHAIN_APPROX_NONE)
    
    detected_rects = []#### ******************************
    
#     img_with_contours = cv.drawContours(frame1.copy(), contours, -1, (0,255,0), 3)
#     x1, y1 = (100,130)
#     width = 400
#     height = frame1.shape[0] - y1
    for cnt in contours:
        x, y, w, h = cv.boundingRect(cnt)
        if x>x1 and y > y1 and x < x1+width and y<y1+height and (cv.contourArea(cnt) >= 700):
            detected_rects.append([x,y,w,h]) ### *************************************
                ############################
            cv.rectangle(original_1, (x,y), (x+w, y+h), (0, 255, 0), 2, cv.LINE_AA)
            
    return detected_rects ### *************************************

#     return cv.rectangle(original_1, (x1, y1), (x1+width, y1+height), (0, 0, 255), 2, cv.LINE_AA)
#     cv.imshow("Rectangles", original_1)
#     key = cv.waitKey(70)
#     if key == 27:
#         break
# cv.destroyAllWindows()

In [5]:
class carTracker:
    def __init__(self):
        self.center_points = {}
        self.id_number = 0
        
        
    def update(self, detected_rects):
        rect_ids = []
        
        for rect in detected_rects:
            x, y, w, h = rect
            cent_x = (2*x + w)//2
            cent_y = (2*y + h)//2
            
            the_same_object = False
            
            for id, pt in self.center_points.items():
                distance = math.hypot(cent_x - pt[0], cent_y - pt[1])
                
                if distance < 40:
                    the_same_object = True
                    self.center_points[id] = (cent_x, cent_y)
                    rect_ids.append([x, y, w, h, id])
                    break
            if the_same_object == False:
                self.center_points[self.id_number] = (cent_x, cent_y)
                rect_ids.append([x, y, w, h, self.id_number])
                self.id_number += 1
                
        new_center_points = {}
        ###########################################
#         for obj in rect_ids:
#             _,_,_,_, obj_id = obj
#             center = self.center_points[obj_id]
# #             print(center)
#             new_center_points[obj_id] = center
##########################################
######################################################
        for obj_id, obj in self.center_points.items():
            if self.center_points[obj_id][1]>500 and (self.center_points[obj_id] not in rect_ids):
                continue
                
            center = self.center_points[obj_id]
            new_center_points[obj_id] = center

#########################################
            
        self.center_points = new_center_points.copy()
        return rect_ids
        

In [6]:
tracker = carTracker() ### *******************************
frame1 = cv.imread(r"D:\Not yours\NU\Master's\2nd semester\Introduction to image processing\Project\Video\frame" + str(i) + ".jpg")

resize_factor = 50
frame1 = resize(frame1, resize_factor)

#Saving a video

w = frame1.shape[1]
h = frame1.shape[0]
fourcc = cv.VideoWriter_fourcc(*'mp4v') 
video = cv.VideoWriter('video.avi', fourcc, 14, (w, h))


for i in range(1, len(frames)-1):
    frame2 = cv.imread(r"D:\Not yours\NU\Master's\2nd semester\Introduction to image processing\Project\Video\frame" + str(i) + ".jpg")
    
    frame2 = resize(frame2, resize_factor)
    x1 = 200
    y1 = 250
    width = 400
    height = frame2.shape[0] - y1 -100
    
    detected_rects = Car_detection(frame1.copy(), frame2.copy(), 50, x1, y1, width, height)
    
    rect_ids = tracker.update(detected_rects)
    for  rect_id in rect_ids:
        x,y,w,h,id = rect_id
        cv.putText(frame1, str(id),(x,y-15), cv.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 2)
        cv.rectangle(frame1, (x,y),(x+w, y+h), (0,255,0),2,  cv.LINE_AA)
    
    
    cv.rectangle(frame1, (x1, y1), (x1+width, y1+height), (0, 0, 255), 2, cv.LINE_AA)
    
    video.write(frame1)
    
    cv.imshow("detected cars", frame1)
    key = cv.waitKey(25)
    frame1 = frame2
    if key & 0xFF == ord('q'):
        break
cv.destroyAllWindows()
video.release()


